In [ ]:
# # python -m spacy download en_core_web_sm
# pd.set_option('display.max_column', None)
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_seq_items', None)
# pd.set_option('display.max_colwidth', 500)
# pd.set_option('expand_frame_repr', True)

In [3]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime

df = pd.read_csv('stock_data.csv')
df['Text'] = df['Text'].astype(str)
df['Date'] = pd.to_datetime(df['Date']).dt.date
df['Sentiment'] = df['Sentiment'].astype(np.int64)
df.sort_values(by='Date',inplace=True)

company = pd.read_csv('company.csv')
check2 = company['Symbol'].apply(str).tolist()
check1 = company['Company Name'].apply(str).tolist()
check3 = company['Company Name'].apply(str).apply(lambda x: (x.split()[0]).lower()).tolist()

# current_date = datetime.date.today()
# start_date = current_date - datetime.timedelta(days=1)

date_string = "2023-06-01"
start_date = datetime.strptime(date_string, "%Y-%m-%d").date()
test_data = df[(df['Date']==start_date)]
test_data.loc[test_data['Sentiment'] == 2, 'Sentiment'] = -1
overall_mean =test_data[test_data['Company'].str.contains('')==True]['Sentiment'].mean()
company_count=test_data['Company'].value_counts()
top_5_companies = company_count.head(5).index
test_data = test_data[test_data['Company'].isin(top_5_companies)]
company_count = test_data.groupby('Company')['Sentiment'].mean()
company_name = company_count.index.tolist()
company_sentiment = company_count.values.tolist()
result = pd.DataFrame({"Company_Name":company_name,"Company_Symbol":company_name,"Company_sentiment":company_sentiment,"Overal_day_Sentiment":[overall_mean]*5,"Result":overall_mean})
def get_company_name(text):
    if text in check3:
        return check1[check3.index(text)]

    elif text in check2:
         return check1[check2.index(text)]

def get_company_symbol(text):
    if text in check1:
        return (check2[check1.index(text)])

result['Company_Name'] = result['Company_Name'].apply(get_company_name)
result['Company_Symbol'] = result['Company_Name'].apply(get_company_symbol)
result['Result'] = np.mean(result[['Company_sentiment','Overal_day_Sentiment']],axis=1)
result

,Company_Name,Company_Symbol,Company_sentiment,Overal_day_Sentiment,Result
0,Adani Enterprises Limited,ADANIENT,0.25,0.615385,0.432692
1,Apollo Hospitals Enterprise Limited,APOLLOHOSP,1.00,0.615385,0.807692
2,Bajaj Finance Limited,BAJFINANCE,1.00,0.615385,0.807692
3,Laurus Labs Limited,LAURUSLABS,1.00,0.615385,0.807692
4,Tata Consultancy Services Limited,TCS,1.00,0.615385,0.807692


In [22]:
import yfinance as yf
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
import math
import datetime as dt
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score 

# Retrieve Adani Group's stock data from Yahoo Finance
adani_stock = yf.download("APOLLOHOSP.NS", start="2005-01-01", end="2023-06-01")

# Preprocess the data
adani_stock['Date'] = adani_stock.index
adani_stock['Date'] = adani_stock['Date'].astype(str)
adani_stock = adani_stock[['Date', 'Close']]
adani_stock.reset_index(drop=True, inplace=True)

# Split the data into train and test sets
train_data, test_data = train_test_split(adani_stock, test_size=0.2, shuffle=False)

# Preprocess the features
scaler = StandardScaler()
train_features = scaler.fit_transform(train_data.index.values.reshape(-1, 1))
test_features = scaler.transform(test_data.index.values.reshape(-1, 1))

# Preprocess the target variable
scaler_y = StandardScaler()
train_target = scaler_y.fit_transform(train_data['Close'].values.reshape(-1, 1))
test_target = scaler_y.fit_transform(test_data['Close'].values.reshape(-1, 1))

# Train the Support Vector Machine (SVM) regression model
svm_model = SVR(kernel='rbf', C=1e2, gamma=0.1)
svm_model.fit(train_features, train_target.ravel())

# Predict the stock prices for train and test data
train_predicted_prices = svm_model.predict(train_features)
test_predicted_prices = svm_model.predict(test_features)

# Inverse transform the predicted prices
train_predicted_prices = scaler_y.inverse_transform(train_predicted_prices.reshape(-1, 1))
test_predicted_prices = scaler_y.inverse_transform(test_predicted_prices.reshape(-1, 1))

# Calculate MSE and R2 scores for train and test data
train_mse = mean_squared_error(train_data['Close'], train_predicted_prices)
test_mse = mean_squared_error(test_data['Close'], test_predicted_prices)

train_r2 = r2_score(train_data['Close'], train_predicted_prices)
test_r2 = r2_score(test_data['Close'], test_predicted_prices)

print("Train data MSE: ", train_mse)
print("Test data MSE: ", test_mse)
print("Train data R2: ", train_r2)
print("Test data R2: ", test_r2)


[*********************100%***********************]  1 of 1 completed
Train data MSE:  7306168.382101518
Test data MSE:  2856887.6188360746
Train data R2:  -36.18162672506455
Test data R2:  -0.6317290930683279


In [17]:
print("Test data explained variance regression score:", explained_variance_score(scaler_y.inverse_transform(test_target.reshape(-1, 1)), predicted_prices))

Test data explained variance regression score: -3.5575289939683055


In [18]:
print("Test data R2 score:", r2_score(scaler_y.inverse_transform(test_target.reshape(-1, 1)), predicted_prices))

Test data R2 score: -10.97293250935929


In [12]:
train_data.index.values

array([   0,    1,    2, ..., 4153, 4154, 4155], dtype=int64)